Propensity Score Matching(Full matching) - Linear Propensity Score

1. Data Processing

In [1]:
import numpy as np
import pandas as pd
import time
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [2]:
highDim = pd.read_csv('highDim_dataset.csv')
lowDim = pd.read_csv('lowDim_dataset.csv')

In [3]:
X_high = highDim.drop(['A','Y'], axis = 1)
X_low = lowDim.drop(['A','Y'], axis = 1)

y_high = highDim[['A']]
y_low = lowDim[['A']]

2. Choosing the best parameter to calculate propensity score

In [149]:
def best_para(data, C):
    X=data.drop(['A','Y'], axis = 1)
    y=data[['A']]
    diff=[]
    for c in C:
        clf = LogisticRegression(penalty='l1', C = c, solver = 'liblinear')
        clf.fit(X, y.values.ravel())
        ps_logit=clf.predict_log_proba(X)[:, 1]
        data['log_ps']=ps_logit
        treated=data[data['A']==1]
        control=data[data['A']==0]
        di=max(treated['log_ps'])-min(treated['log_ps'])
        dj=max(control['log_ps'])-min(control['log_ps'])
        diff.append(abs(di-dj))
    best_ind=diff.index(min(diff))
    best_c=C[best_ind]
    best_diff=diff[best_ind]
    return best_c, best_diff

In [320]:
#C = [1, 0.9, 0.8, 0.7, 0.6, 0.5, 0.4, 0.3, 0.2, 0.1]

#best_para(lowDim, C=C)
#best_low=0.2

#best_para(highDim, C=C)
#best_high=0.5

3. Propensity score matching model

In [52]:
def PSM(treated_df, control_df):
    
    treated_df.loc[:,'group']=None
    treated_df.loc[:,'control_Y']=None
    
    diff_i=[]
    for i in range(len(treated_df)):
        diff_j=[]
        for j in range(len(control_df)):
            diff_j.append(abs(control_df.loc[j,'log_ps']-treated_df.loc[i,'log_ps']))

        ind_j=diff_j.index(min(diff_j))
        treated_df.loc[i,'control_Y']=control_df.loc[ind_j,'Y']
        diff_i.append(min(diff_j))
        
    r=(max(diff_i)-min(diff_i))/5
    for i in range(len(treated_df)):
        if diff_i[i] < min(diff_i)+r:
            treated_df.loc[i,'group']=1
        elif diff_i[i] >= min(diff_i)+r and diff_i[i] < min(diff_i)+r*2:
            treated_df.loc[i,'group']=2
        elif diff_i[i] >= min(diff_i)+r*2 and diff_i[i] < min(diff_i)+r*3:
            treated_df.loc[i,'group']=3
        elif diff_i[i] >= min(diff_i)+r*3 and diff_i[i] < min(diff_i)+r*4:
            treated_df.loc[i,'group']=4
        else:
            treated_df.loc[i,'group']=5
    
    ATE=0
    for k in range(treated_df.loc[:,'group'].max()):
        group=treated_df[treated_df.loc[:,'group']==k+1]
        if len(group)!=0:
            ATE=ATE+(group.loc[:,'Y']-group.loc[:,'control_Y']).mean()*len(group)/len(treated_df)
    
    return ATE

4. Performance on Low Dimension Data

In [53]:
start_low = time.time()
clf_low = LogisticRegression(penalty='l1', C = 0.2, solver = 'liblinear')
clf_low.fit(X_low, y_low.values.ravel())
ps_logit_low = clf_low.predict_log_proba(X_low)[:, 1]
lowDim['log_ps']=ps_logit_low

In [54]:
treated_low = lowDim[lowDim['A']==1]
treated_low = treated_low.reset_index(drop = True)
control_low = lowDim[lowDim['A']==0]
control_low = control_low.reset_index(drop = True)

PSM(treated_low, control_low)

2.5150571697333914

In [55]:
time.time()-start_low

12.585533857345581

5. Performance on High Dimension Data

In [56]:
start_high = time.time()
clf_high = LogisticRegression(penalty='l1', C = 0.5, solver = 'liblinear')
clf_high.fit(X_high, y_high.values.ravel())
ps_logit_high = clf_high.predict_log_proba(X_high)[:, 1]
highDim['log_ps']=ps_logit_high

In [57]:
treated_high = highDim[highDim['A']==1]
treated_high = treated_high.reset_index(drop = True)
control_high = highDim[highDim['A']==0]
control_high = control_high.reset_index(drop = True)

PSM(treated_high, control_high)

-3.0123405906880065

In [58]:
time.time()-start_high

27.20747995376587